In [1]:
import os
os.chdir('..')

# Import Files

In [ ]:
from src.agent_interface import *
from src.agent import *
from src.env import *
from src.price_data import *
from src.prior_measure import *
from src.q import *
from src.robust import *
from src.train import *
from src.util import *

# Resume Training

In [ ]:
checkpoint_file_name = "checkpoint_ep5.pt"
runtime_folder = "20260227_130436_3"
model_name = "PORDQN_TRANSACTION0025_CLIPGRADIENTFALSE_EPSSCHEDULER"

In [ ]:
c = Config()
config_path = c.build_config_path(model_name)
stock_params, env_params, training_controller_params, duality_params, q_params, dqn_params, eps_scheduler_params, other_params = c.load_config(config_path)

In [ ]:
yf = YahooFinance(**stock_params)
df = yf.pipeline()
asset_log_returns = df['log_return'].dropna().to_numpy()
env = PortfolioEnv(asset_log_returns=asset_log_returns, **env_params)

[*********************100%***********************]  1 of 1 completed


In [ ]:
device = torch.device('cuda')
model_name, n_epochs = other_params.values()

training_controller = TrainingController(**training_controller_params)
prior_measure = PriorStudentDistribution(device=device)
duality_operator = DualityHQOperator(**duality_params)
q = QFunc(**q_params)
writer = PORDQNProgressWriter(model_name, timestamp_folder_name=runtime_folder, overwrite_existing_checkpoint_file=True)
eps_scheduler = EpsilonGlobalScheduler(**eps_scheduler_params)
agent = PORDQN(action_values=env.action_values, training_controller=training_controller, prior_measure=prior_measure, duality_operator=duality_operator,
               epsilon_scheduler=eps_scheduler, qfunc=q, writer=writer, device=device, **dqn_params)

In [ ]:
load_model = LoadModel()
agent, current_epoch = load_model.restore_agent_state(agent, model_name, checkpoint_file_name, runtime_folder, target_device=device)

In [ ]:
cum_rewards = train_agent(env, agent, current_epoch=6, n_epochs=10, writer=writer)

Episode 3: 100%|██████████| 7491/7491 [2:04:26<00:00,  1.00it/s]  


Episode 3 mean of summed rewards: -99.9750%


Episode 4: 100%|██████████| 7491/7491 [1:55:40<00:00,  1.08it/s]


Episode 4 mean of summed rewards: -99.6951%


Episode 5: 100%|██████████| 7491/7491 [2:38:48<00:00,  1.27s/it]  


Episode 5 mean of summed rewards: -95.3760%
